## Flask-OAuthlib
extension for Flask that provides support for OAuth authentication. OAuth allows users to grant access to their data to third-party applications without sharing their passwords. This is often used for social media logins (e.g., logging in with Google or Facebook).

**OAuth 2.0** is the most commonly used version, which involves an authorization flow where a user is redirected to a third-party service (e.g., Google) to log in, and the application retrieves an access token to interact with the service on the user's behalf.

In [ ]:
# Installation
!pip install Flask-OAuthlib

In [ ]:
from flask import Flask, redirect, url_for, session, request
from flask_oauthlib.client import OAuth

app = Flask(__name__)
app.secret_key = 'random_secret_key'
oauth = OAuth(app)  # Creating the OAuth object that will handle OAuth 2.0 interactions.

### OAuth Providers
<table>
    <thead>
        <tr>
            <th>Provider</th>
            <th>OAuth URL</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>Google OAuth</td>
            <td><a href="https://developers.google.com/identity/protocols/oauth2" target="_blank">Google OAuth</a></td>
        </tr>
        <tr>
            <td>Facebook OAuth</td>
            <td><a href="https://developers.facebook.com/docs/facebook-login/" target="_blank">Facebook OAuth</a></td>
        </tr>
        <tr>
            <td>GitHub OAuth</td>
            <td><a href="https://docs.github.com/en/developers/apps/building-oauth-apps" target="_blank">GitHub OAuth</a></td>
        </tr>
        <tr>
            <td>Twitter OAuth</td>
            <td><a href="https://developer.twitter.com/en/docs/authentication/oauth-2-0" target="_blank">Twitter OAuth</a></td>
        </tr>
        <tr>
            <td>LinkedIn OAuth</td>
            <td><a href="https://www.linkedin.com/developers/docs/oauth-2-0" target="_blank">LinkedIn OAuth</a></td>
        </tr>
        <tr>
            <td>Microsoft OAuth</td>
            <td><a href="https://learn.microsoft.com/en-us/azure/active-directory/develop/v2-oauth2-auth-code-flow" target="_blank">Microsoft OAuth</a></td>
        </tr>
        <tr>
            <td>GitLab OAuth</td>
            <td><a href="https://docs.gitlab.com/ee/administration/oauth.html" target="_blank">GitLab OAuth</a></td>
        </tr>
        <tr>
            <td>Slack OAuth</td>
            <td><a href="https://api.slack.com/authentication/oauth-v2" target="_blank">Slack OAuth</a></td>
        </tr>
        <tr>
            <td>Spotify OAuth</td>
            <td><a href="https://developer.spotify.com/documentation/general/guides/authorization-guide/" target="_blank">Spotify OAuth</a></td>
        </tr>
        <tr>
            <td>Twitch OAuth</td>
            <td><a href="https://dev.twitch.tv/docs/authentication/getting-tokens-oauth" target="_blank">Twitch OAuth</a></td>
        </tr>
        <tr>
            <td>Yahoo OAuth</td>
            <td><a href="https://developer.yahoo.com/oauth/" target="_blank">Yahoo OAuth</a></td>
        </tr>
        <tr>
            <td>Amazon OAuth</td>
            <td><a href="https://developer.amazon.com/docs/login-with-amazon/overview.html" target="_blank">Amazon OAuth</a></td>
        </tr>
        <tr>
            <td>Reddit OAuth</td>
            <td><a href="https://www.reddit.com/dev/api/" target="_blank">Reddit OAuth</a></td>
        </tr>
        <tr>
            <td>Instagram OAuth</td>
            <td><a href="https://developers.facebook.com/docs/instagram-basic-display-api" target="_blank">Instagram OAuth</a></td>
        </tr>
    </tbody>
</table>

### Setting Up OAuth with Google

In [ ]:
google = oauth.remote_app(
    'google',
    consumer_key='GOOGLE_CONSUMER_KEY',  # Replace this with your Google API client ID.
    consumer_secret='GOOGLE_CONSUMER_SECRET',  # Replace with your Google API client secret.
    request_token_params={
        'scope': 'email',  # This defines what access we want to request, in this case, email.
    },
    base_url='https://www.googleapis.com/oauth2/v1/',
    request_token_url=None,  # Not needed for OAuth 2.0.
    access_token_method='POST',  # OAuth 2.0 uses POST method for token exchange.
    access_token_url='https://accounts.google.com/o/oauth2/token',
    authorize_url='https://accounts.google.com/o/oauth2/auth',  # The URL to which users are redirected to authorize access.
)

### Setting Up Template

In [ ]:
<body>
    <h1>Login with Google</h1>
    <a href="{{ url_for('login') }}">Login with Google</a>
</body>

### Setting Up Routes for OAuth Flow

- **Login Route**: When the user accesses /login, they will be redirected to Google’s OAuth authorization page. If the user successfully logs in, Google will redirect them back to our callback route.

In [ ]:
@app.route('/login')
def login():
    # Redirects the user to Google’s OAuth authorization page
    return google.authorize(callback=url_for('authorized', _external=True))

- **Callback Route**: After the user logs in on Google’s OAuth page, they will be redirected back to our /login/authorized route, which handles receiving the authorization code from Google and exchanging it for an access token.

In [ ]:
@app.route('/login/authorized')
def authorized():
    # Google sends the response containing the authorization code
    response = google.authorized_response()  # This will contain the access token.
    
    # If authorization failed (no access token is returned), show an error message.
    if response is None or response.get('access_token') is None:
        return 'Access denied: reason={} error={}'.format(
            request.args['error_reason'],
            request.args['error_description']
        )

    # Store the access token in the session
    session['google_token'] = (response['access_token'], '')  # You can store this token in a database for long-term use.
    
    # Use the access token to fetch user data (e.g., email)
    user_info = google.get('userinfo')
    return 'Logged in as ' + user_info.data['email']


- **Token Getter**: This function will be used to retrieve the access token from the session when making future API calls on behalf of the user.

In [ ]:
@google.tokengetter
def get_google_oauth_token():
    return session.get('google_token')

### How It Works:
- The user visits the **/login** route, which triggers a redirection to Google’s OAuth authorization page.
- Once the user logs in and grants access, they are redirected back to your app, where the authorization code is exchanged for an access token.
- The access token is stored in the session and can be used to make requests to Google’s API on the user’s behalf (e.g., getting their email).